In [1]:
import numpy as np
import random as rnd
import openpyxl as opx
from scipy.stats import norm
import time

In [2]:
def gaussian_distribution(days_distribution):
    # Parámetros de la campana de Gauss deseada
    mean = days_distribution / 2
    std_dev = 8.8

    # Crear un rango de valores en el eje x
    x_values = np.linspace(1, days_distribution, days_distribution)
    y_values = norm.pdf(x_values, mean, std_dev)

    # Ajustar la campana de Gauss para que la suma de las probabilidades sea 1
    sum_y_values = np.sum(y_values)
    ajuste = (1 - sum_y_values) / days_distribution

    for i in range(len(y_values)):
        y_values[i] += ajuste

    #print("Distribución Objetivo:\n", y_values)c
    #suma = np.sum(y_values)
    #print("Suma:", suma)

    return y_values, x_values

In [3]:
# Función de inicialización de la población
def initialize_population(population_size, genome_size, remainder):
    # Crear un arreglo con valores entre 0 y 0.5
    population = [np.random.uniform(0, 0.5, genome_size) for _ in range(population_size)]

    # Ajustar algunos valores para que la suma sea igual al remanente
    for i in range(population_size):
        population[i] = population[i] / np.sum(population[i]) * remainder

    #print(np.array(population))
    #print("Suma de cada fila:", np.sum(population, axis=1))
    return np.array(population)


In [4]:
def mse_fitness(y_true, y_pred):
  # Calcula la diferencia entre los valores reales y predichos
  error = y_true - y_pred

  # Calcula el cuadrado del error
  squared_error = np.square(error)

  # Calcula la media del cuadrado del error
  mse = (1 / (1 + np.mean(squared_error)))
  return mse

In [5]:
# Cruce de un individuos 
def one_point_crossover(parent1, parent2, crossover_rate):
    if np.random.rand() < crossover_rate:
        crossover_point = np.random.randint(1, len(parent1) - 1)
        #print("Punto de cruce:", crossover_point)
        child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
        child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
        return child1, child2
    else:
        return parent1, parent2

In [6]:
# Mutación de un individuo
def mutate(individual, mutation_rate):
    mask = np.random.rand(len(individual)) < mutation_rate
    individual[mask] = np.random.rand(sum(mask))
    return individual

In [7]:
# Algoritmo genético
def genetic_algorithm(target_distribution, real_progress, population_size=50, generations=50, crossover_rate=0.8, mutation_rate=0.1, elitism_rate=0.1):
    progress_days = sum(1 for value in real_progress if value is not np.nan)
    genome_size = len(target_distribution) - progress_days
    # Eliminamos los días que ya han pasado
    target_dist_to_end = target_distribution[progress_days:]
    
    # Inicializar la población arreglos que suman el remanente para terminar las tareas
    remainder = 1 - sum(value for value in real_progress if value is not np.nan)
    population = initialize_population(population_size, genome_size, remainder)
    # Calcular la aptitud de cada individuo en la población
    fitness_values = np.array([mse_fitness(individual, target_dist_to_end) for individual in population])
    evolution = []

    for generation in range(generations):
        elite_count = int(population_size * elitism_rate)
        elite_indices = np.argsort(fitness_values)[::-1][:elite_count]
        #print(fitness_values[np.argsort(fitness_values)])
        #print("2", fitness_values[elite_indices])
        new_generation = population[elite_indices]

        while(len(new_generation) < population_size):
            #Selección por torneo de 16 individuos
            selected_indices = rnd.sample(range(len(fitness_values)), 16)
            
            while len(selected_indices) > 2:
                #print(selected_indices)
                indx = []

                for i in range(len(selected_indices) // 2):
                    #print(fitness_values[selected_indices[i]], fitness_values[selected_indices[-(i + 1)]])
                    if fitness_values[selected_indices[i]] > fitness_values[selected_indices[-(i + 1)]]:
                        indx.append(selected_indices[i])
                    else:
                        indx.append(selected_indices[-(i + 1)])
                
                selected_indices = indx

            
            parents = population[selected_indices]
            #print("Padres:", parents)   

            # Cruzar y mutar para obtener nuevos individuos
            children = [mutate(child, mutation_rate) for child in one_point_crossover(*parents, crossover_rate)]
            #print("Hijos:", children)
            new_generation = np.concatenate((new_generation, children))
            #print("xxx",  new_generation, "xxx")

        population = new_generation[:population_size]
        fitness_values = np.array([mse_fitness(individual, target_dist_to_end) for individual in population])

        # Imprimir la aptitud del mejor individuo en esta generación
        best_fitness = np.max(fitness_values)
        evolution.append({"Generación": generation + 1, "Mejor Aptitud": best_fitness})
        #print("Generación:", generation + 1, "Mejor Aptitud", best_fitness)

    
    # Devolver el mejor individuo al final del algoritmo
    best_index = np.argmax(fitness_values)
    best_individual = population[best_index]

    data_convergence = [{"Best_Fitness": best_fitness, "Generations": generations, "Population": population_size, "Crossover_Rate": crossover_rate, "Mutation_Rate": mutation_rate, "Elitism_Rate": elitism_rate}]
    
    return best_individual, data_convergence, evolution

In [8]:
# Crear un libro de trabajo en blanco
wb = opx.Workbook()

# Agregar una hoja (por defecto ya hay una hoja activa)
hoja_activa = wb.active
hoja_activa.title = "General"  # Cambia el nombre de la hoja si lo deseas

#Ejecucipon de la tarea
work_days = 44
pruebas = 21
# Se crea una simulación de la distribución de probabilidad de la campana de Gauss para una tarea planificada.
target_distribution, x_values = gaussian_distribution(work_days)

# Simulamos un avance real de la tarea que representa un atraso del 20%
real_progress = np.array([target_distribution[i] * 0.8 if i < 10 else None for i in range(work_days)])

hoja_activa['A1'] = "Test"
hoja_activa['B1'] = "Best Fitness"
hoja_activa['C1'] = "Generations"
hoja_activa['D1'] = "Population"
hoja_activa['E1'] = "Crossover Rate"
hoja_activa['F1'] = "Mutation Rate"
hoja_activa['G1'] = "Elitism Rate"
hoja_activa['H1'] = "Time"

# Ejecutar el algoritmo genético
for i in range(pruebas):
    #Números aleatorios para generaciones
    gen = rnd.randint(20, 10000)
    #Números aleatorios para la población
    pop = rnd.randint(16, 10000)
    #Números aleatorios para cruce
    cru = round(rnd.uniform(0.6, 1), 4)
    #Números aleatorios para mutación
    mut = round(rnd.uniform(0.001, 0.1), 4)
    #Números aleatorios para elitismo
    eli = round(rnd.uniform(0.0, 0.1), 4)

    inicio = time.time()
    #Ejecutar el test para obtener los parámetros del algoritmo genético
    best_solution, data_convergence, evolution = genetic_algorithm(target_distribution, real_progress, population_size=pop, generations=gen, crossover_rate=cru, mutation_rate=mut, 
                                                                   elitism_rate=eli)
    #Ejecutar el algoritmo genético con los paramatros obtenidos del test
    #best_solution, data_convergence, evolution = genetic_algorithm(target_distribution, real_progress, population_size=pop, crossover_rate=cru, mutation_rate=mut)
    fin = time.time()
    tiempo_ejecucion = fin - inicio
    hoja_activa.append([str(i + 1), data_convergence[0]["Best_Fitness"], data_convergence[0]["Generations"], data_convergence[0]["Population"], data_convergence[0]["Crossover_Rate"], data_convergence[0]["Mutation_Rate"], data_convergence[0]["Elitism_Rate"], round(tiempo_ejecucion, 4)])
    # Medición del tiempo de ejecución
    
    evolution_sheet = wb.create_sheet("Test" + str(i + 1))
    evolution_sheet.append(["Generación", "Mejor Aptitud T" + str(i + 1)])

    for e in evolution:
        evolution_sheet.append([e["Generación"], e["Mejor Aptitud"]])
    
    print(f"Test {i + 1:02d} - Best Fitness: {data_convergence[0]['Best_Fitness']:.16f}, Generations: {data_convergence[0]['Generations']:04d}, Population: {data_convergence[0]['Population']:04d}, Crossover Rate: {data_convergence[0]['Crossover_Rate']:.4f}, Mutation Rate: {data_convergence[0]['Mutation_Rate']:.4f}, Elitism Rate: {data_convergence[0]['Elitism_Rate']:.4f}, Time: {round(tiempo_ejecucion, 4):02f}")

wb.save(f"AGTest10.xlsx")


Test 01 - Best Fitness: 0.9999840671962357, Generations: 0339, Population: 0178, Crossover Rate: 0.8541, Mutation Rate: 0.0911, Elitism Rate: 0.0920, Time: 0.855300
Test 02 - Best Fitness: 0.9999943150563444, Generations: 0470, Population: 0195, Crossover Rate: 0.9444, Mutation Rate: 0.0746, Elitism Rate: 0.0966, Time: 1.307500
Test 03 - Best Fitness: 0.9999981242072705, Generations: 0935, Population: 0340, Crossover Rate: 0.8482, Mutation Rate: 0.0914, Elitism Rate: 0.0919, Time: 4.646300
Test 04 - Best Fitness: 0.9999991640905124, Generations: 0654, Population: 0522, Crossover Rate: 0.9940, Mutation Rate: 0.0703, Elitism Rate: 0.0965, Time: 5.112600
Test 05 - Best Fitness: 0.9999990569877164, Generations: 0644, Population: 0596, Crossover Rate: 0.7972, Mutation Rate: 0.0775, Elitism Rate: 0.0901, Time: 5.699400
Test 06 - Best Fitness: 0.9999979980643473, Generations: 0354, Population: 0538, Crossover Rate: 0.8519, Mutation Rate: 0.0694, Elitism Rate: 0.0953, Time: 2.814300
Test 07 - 